<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/lec_title.png?raw=true" alt="2019年度ゲノム情報解析入門" height="100px" align="middle">

In [0]:
"""
＊重要＊
最初にこのセルを実行してください。
この実習で使うサンプルファイルをダウンロードします。
"""
!wget https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_bulk.txt -O mutmap_bulk.txt

# Pythonライブラリを使う -実践-

## 大規模データ処理

- SNP-indexを求める
- SNP-indexのグラフを描く
- Sliding window解析（発展）


テーブル状に整ったデータであれば、前回の講義で学習したpandasライブラリを利用することで、より高速かつ簡単なデータ処理が可能になります。

今回の実習では、

1.pandasを使ってデータ処理（SNP-indexの計算）をしてみましょう。

2.さらに、Matplotlibライブラリを利用して、データをグラフにしてみましょう。

それではまずpandasデータフレームとしてファイルを読み込みましょう。

In [0]:
import pandas as pd   # pandasを読み込む

dataset = 'mutmap_bulk.txt'   # 読み込みファイル名    

df = pd.read_csv(dataset, sep="\t", header=None, names=['chr', 'pos', 'ref_nucl', 'alt_nucl', 'ref_N', 'alt_N'])   # ファイルを読み込む
# header=None（列名行なし）を指定した場合、列名として通し番号が付けられます。
# names=[リスト]とすることで、通し番号の代わりに列名を指定することが可能です。

df   # 表示

今回の講義では、MutMap (Abe et al., 2012)の論文のシーケンスを解析したデータを使います。
MutMapとは、遺伝子マッピング手法の一つです（より詳しく知りたい方は、下記の論文をご覧ください）。

Abe, A., Kosugi, S., Yoshida, K., Natsume, S., Takagi, H., Kanzaki, H., Matsumura, H., Yoshida, K., Mitsuoka, C., Tamiru, M., Innan, H., Cano, L., Kamoun, S., Terauchi, R. (2012). [Genome sequencing reveals agronomically important loci in rice using MutMap. Nature biotechnology, 30(2), 174.](https://www.nature.com/articles/nbt.2095)

MutMapは次のような手順で解析を行います。

- 1.突然変異系統とその元となった系統（オリジナル系統）を交配する。
- 2.交配後第二世代（F2集団）を作成し、突然変異形質を持つ個体のDNAを集める（以下バルクDNA）。
- 3.オリジナル系統のDNAとバルクDNAをシーケンスする。
- 4.両者のシーケンスを比較すると、突然変異の位置と、オリジナル系統の塩基や突然変異の塩基の種類がわかります。また、それぞれの塩基がいくつあるか（オリジナル系統と同じ塩基の個数と突然変異系統と同じ塩基の個数）といった情報も得られます。
- 5.突然変異形質を持つF2個体は、ゲノム上に散在する変異のうち、形質に関わる変異を共通で持っているはずです。そのため、形質に関わる変異箇所では、突然変異系統の塩基のみ検出されることが期待されます。
- 6.そこで、変異箇所すべてについて、突然変異塩基の割合（「SNP-index」と呼ばれるもの）を調べます。
- 7.調べた結果をグラフに描くことで、形質に関わる原因変異を特定または絞り込みができます。

今回調べる形質は、イネの葉が「淡緑色（pale green）」となる形質です。

サンプルデータとして、この形質のMutMap解析のデータを扱います。
データはテキストファイルで,、次のような情報が並んでいます。

#1.SNP-indexを求める

今回の講義では、前回学習したpandasライブラリを利用し、SNP-indexという数値の計算をしてみましょう。

### SNPとは
SNP（Single Nucleotide Polymorphism、一塩基多型）とは、個体間においてゲノムDNA上の1塩基が異なる現象です。

### SNP-indexとは
SNP-indexとは、染色体上の特定の領域が生物の形質の違いにどれだけ関わっているのかを示す指標です。

### 計算方法

計算に使う列を取り出し、計算式に直接入れる。

### SNP-index計算式
```python
SNP-index = alt_N / (ref_N + alt_N)
```



# 実習1

SNP-indexの計算式を表すコードの続きを書いて、SNP-indexを表示させてください。

### 解答例1

In [0]:
df['snp_index'] = df['alt_N'] / (df['ref_N'] +df['alt_N'])

df

# 2.条件に合うデータを抽出する（前回の復習）

前回、条件に合うデータの抽出方法を学習しました。
おさらいですが、コードの書き方は以下の通りです。

```python
データフレーム[ (条件式)]   # 条件式が一つの場合、丸括弧()は省略可能です

データフレーム[ (条件式1) & (条件式2)]   # 条件2つ（AND条件）
データフレーム[ (条件式1) | (条件式2)]   # 条件2つ（OR条件）
```
確認として例を見てみましょう。
次のセルを実行してみてください。


In [0]:
df[ df['ref_nucl']=='A' ]

df['ref_nucl']=='A'で、条件に合うデータをTrue、合わないデータをFalseと認識したリスト状のデータが得られます。つまりこの場合は、ref_nuclの塩基がAのものはTrue、A以外のものはFalseと認識されています。

さらに、df[ (True/Falseリスト) ]の形にすることで、Trueのデータのみを表示させています。つまり上記のコードでは、ref_nuclの塩基がAのものだけが表示されることになります。

# 実習2

それでは、SNP-indexが0.8以上のものを取り出すコードを書いて、表示させてください。

### 解答例2

In [0]:
df[df['snp_index']>=0.8]

# 3.SNP-indexのグラフを描く

それでは、Pythonのグラフ描画で有名なライブラリ Matplotlib を使ってグラフを描いてみましょう。

Matplotlibについては前回の講義で学習したかと思います。

グラフ描写の基本は、「透明なシート（レイヤー）を重ねて、ひとつのグラフを作っていく」です。

今日の実習では、x軸に染色体上の位置、y軸にSNP-index（「1. SNP-indexを計算する」で求めた）をとったグラフ（散布図）を作成してみましょう。


In [0]:
import matplotlib.pyplot as plt   # ライブラリを読み込む

df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])   # SNP-indexの計算式

x = df['pos']         # x軸データ
y = df['snp_index']   # y軸データ

fig = plt.figure(figsize=[16,9])    # グラフフィールドの設定
plt.scatter(x, y, color='gray')      # 散布図
plt.title('SNP-index on chromosome 10', fontsize=24)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=16)           # y軸ラベル

# 実習3

それでは、上記のコードに、「SNP-indexが0.8より大きいものを赤色にする」というコードを書き足して、表示させてください。

ヒントが緑色の文字で書かれているので参考にしてください。

In [0]:
import matplotlib.pyplot as plt  

df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])  

x = df['pos']        
y = df['snp_index']   

fig = plt.figure(figsize=[16,9])    
plt.scatter(x, y, color='gray') 
plt.title('SNP-index on chromosome 10', fontsize=24)
plt.xlabel('Position (x 10 Mb)', fontsize=16)
plt.ylabel('SNP-index', fontsize=16)           

df1 = df[ df['snp_index'] >= 0.8 ]   
# SNP-indexが0.8より大きいものを表示（"df"のままだと上記のコードが上書きされてしまうことに注意！）

x1 = df1[]         # x軸データを[]内に書き足す
y1 = df1[]   # y軸データを[]内に書き足す

plot.scatter()   # 赤色で表示させるためのコードを()内に書き足す

### 解答例3

In [0]:
import matplotlib.pyplot as plt   

df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])   

x = df['pos']
y = df['snp_index']  

fig = plt.figure(figsize=[16,9])   
plt.scatter(x, y, color='gray')      
plt.title('SNP-index on chromosome 10', fontsize=24)  
plt.xlabel('Position (x 10 Mb)', fontsize=16)  
plt.ylabel('SNP-index', fontsize=16)           

df2 = df[ df['snp_index'] >= 0.8 ]   
# SNP-indexが0.8より大きいものを表示（"df"のままだと上記のコードが上書きされてしまうことに注意！）

x1 = df2['pos']         # x軸データ
y1 = df2['snp_index']   # y軸データ

plt.scatter(x1, y1, color='red') # 赤色で表示


# 4.Sliding Window解析（発展）

MutMap解析では、SNP-indexの区間平均をゲノムに全体にわたって調べる解析（Sliding Window解析）というのをおこない、形質にかかわるゲノム領域を特定します。

形質にかかわる領域付近では、直接の原因となる塩基座位だけでなく、周囲の塩基座位のSNP-indexも1に近づきます。

ここでは上の図のように、Chromosome 10上のSNP-indexの推移を調べてみましょう。
あらかじめプログラムを書いています。

### 解析方法

SNP-indexのSliding Window解析は次のような手順で解析します。

    1.区間サイズとステップサイズを決める。
    2.各区間の代表値（ここでは区間の中央値）用のリストと、区間内の平均SNP-index用のリストを用意する。
    3.while文を使って全ての区間を調べていく。
        区間内に含まれるデータを抽出する。
        抽出したデータからSNP-indexの平均値を算出する。
        区間の中央値と算出した平均SNP-indexを、それぞれのリストに追加する。
        ゲノムの全長を全て調べたらwhileから出る
    4.x軸に中央値、y軸に平均SNP-indexの値をとったグラフ（折れ線グラフなど）を作成する。


In [0]:
###### Sliding Window解析 ######
#---  NumPyライブラリの読み込み ---
import numpy as np

#--- 区間サイズとステップサイズ ---
CHROM_SIZE = 23207287       # Chromosome 10　の全長 (bp)
WIN_SIZE       = 1 * 1000 * 1000     #  1 Mb = 1000 kb = 1,000,000 bp
STEP_SIZE     = 0.2 * 1000* 1000     #  0.2 Mb = 200 kb = 200,000 bp

#--- 区間データ用のリストを準備 ---
win_position  = []  # 区間中央値用リスト
win_snpindex = []  # 平均SNP-index用リスト

#--- 全ての区間を調べる ---
"""
/// 調べる区間 ///
start, end
0, 0+1000 (kb)
200, 200+1000
400, 400+1000
  .
  .
  .

/// WIN_SIZEとSTEP_SIZEを使って表現すると... ///
繰り返し数:　n = 0, 1, 2, ...

start = STEP_SIZE * n  
end = start + WIN_SIZE


end > CHROM_SIZEになったとき、繰り返しを終える。

このように、繰り返し数を指定しない場合は、for文ではなく、while文を使用します。
"""

n = 0 # 繰り返し数
while True:
    
    #--- 区間のstartとend position ---
    start = STEP_SIZE * n 
    end   = start + WIN_SIZE
    
    #--- 区間の中央値をリストに追加する ---
    p = (start + end) / 2
    win_position.append(p)
    
    #--- 区間内データを抽出 ---
    sub = df[(df['pos'] >= start) & (df['pos'] < end)]
    
    #--- SNP-indexの平均値を算出 ---
    i = sub['snp_index'].mean()
    win_snpindex.append(i)
        
    #--- 繰り返し数を+1 ---
    n += 1
    
    #--- 全ての区間を調べた時、whileから出る ---
    if end > CHROM_SIZE:
        break

#--- グラフ作成  ---
fig = plt.figure(figsize=[16,9])
plt.scatter(x, y, color='gray')      # 全データ
plt.title('SNP-index on chromosome 10', fontsize=24)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=16)        # y軸ラベル

#--- グラフにsnp_index>=0.8のデータを重ね描き ---
df2 = df[ df['snp_index'] >= 0.8 ]
x1 = df2['pos']
y1 = df2['snp_index']
plt.scatter(x1, y1, color='red')

#--- 平均SNP-indexの折れ線グラフ（重ね描き） ---
plt.plot(win_position, win_snpindex, color='blue')

さらに、今回は、平均SNP-indexが0.8以上の区間を原因遺伝子の候補領域として取り出してみましょう。

In [0]:
### 平均SNP-indexが0.8以上の区間を取り出す ###

#--- Sliding windowのデータをpandasデータフレームにする ---
sw = pd.DataFrame({ 'pos': win_position, 'snp_index': win_snpindex})

#--- 区間のstartとendの位置 ---
sw['start'] = sw['pos'] - WIN_SIZE / 2 
sw['end'] = sw['pos'] + WIN_SIZE / 2

#--- 平均SNP-indexが0.8以上の区間 ---
sw[sw['snp_index'] >= 0.8]

今回は、「平均SNP-indexが0.8以上」という根拠のないあいまいな設定で候補領域を取り出してみましたが、実際には22,482,910bpの辺りに Chlorophyllide a oxygenase (Os10t0567400) という遺伝子が存在しています。
[MutMapの論文(Abe et al., 2012)](https://www.nature.com/articles/nbt.2095)で、その遺伝子が、実際に「葉の色（緑色/淡緑色）」に関わっている証拠も得られており、突然変異形質の原因遺伝子と考えられています。